In [1]:
import numpy as np
from astropy.table import Table

In [2]:
from causalflow import causalflow, support

# load some demo data

In [3]:
fema = Table.read('../dat/cano2024_dataset.hdf5')

In [4]:
control = (fema['communityRatingSystemDiscount'] == 11.)
treated = ~control

In [5]:
columns = ['amountPaidOnTotalClaim_per_policy', # outcome
           'mean_rainfall', 'avg_risk_score_all', 'median_household_income', 'population', # covariates
           'renter_fraction', 'educated_fraction', 'white_fraction']

In [6]:
control_data = np.vstack([np.ma.getdata(fema[col][control].data) for col in columns]).T[::10]
control_data[:,0] = np.log10(control_data[:,0])
control_data[:,3] = np.log10(control_data[:,3])
control_data[:,4] = np.log10(control_data[:,4])

treated_data = np.vstack([np.ma.getdata(fema[col][treated].data) for col in columns]).T[::10]
treated_data[:,0] = np.log10(treated_data[:,0])
treated_data[:,3] = np.log10(treated_data[:,3])
treated_data[:,4] = np.log10(treated_data[:,4])

# Scenario A

In [7]:
Cflow = causalflow.CausalFlowA()

In [8]:
Cflow.train_support_control(control_data[:,1:], clip_max_norm=1., verbose=True)

Ntrain= 3928, Nvalid= 437


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

In [9]:
Cflow.train_support_treated(treated_data[:,1:], clip_max_norm=1., verbose=True)

Ntrain= 2758, Nvalid= 307


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

In [8]:
Cflow.train_flow_treated(treated_data[:,0], treated_data[:,1:], outcome_range=[[-1.], [6.]])

/opt/anaconda3/envs/noah/lib/python3.7/site-packages/sbi/neural_nets/flow.py:62: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn("In one-dimensional output space, this flow is limited to Gaussians")


 Neural network successfully converged after 23 epochs.

In [ ]:
Cflow.train_flow_control(control_data[:,0], control_data[:,1:], outcome_range=[[-1.], [6.]])

In [ ]:
X_random = control_data[374,1:]
print(Cflow.CATE(X_random, Nsample=10000, progress_bar=True, transf=lambda x: 10**x))

# Scenario B

In [7]:
Cflow = causalflow.CausalFlowB('control')

In [8]:
# train support for base sample
Cflow.train_support(control_data[:,1:], nhidden=128, nblock=5, clip_max_norm=1., verbose=True)

Ntrain= 3928, Nvalid= 437


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

In [9]:
Cflow.train_flow(control_data[:,0], control_data[:,1:], outcome_range=[[-1.], [6.]])

/opt/anaconda3/envs/noah/lib/python3.7/site-packages/sbi/neural_nets/flow.py:62: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn("In one-dimensional output space, this flow is limited to Gaussians")


 Neural network successfully converged after 25 epochs.

In [10]:
X_other = treated_data[:10,1:]
Y_other = treated_data[:10,0]

In [11]:
Cflow.CATE(X_other, Y_other, Nsample=10000, progress_bar=False, transf=lambda x: 10**x)

-4403.6550728752345

# Scenario C

In [12]:
Cflow = causalflow.CausalFlowC()

In [15]:
fake_treatment = np.random.uniform(low=0., high=1., size=control_data.shape[0])[:,None]

In [19]:
# train support for base sample
Cflow.train_support(control_data[:,1:], fake_treatment, nhidden=128, nblock=5, clip_max_norm=1., verbose=True)

Ntrain= 3928, Nvalid= 437


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

In [24]:
Cflow.train_flow(control_data[:,0], control_data[:,1:], fake_treatment, outcome_range=[[-1.], [6.]])

/opt/anaconda3/envs/noah/lib/python3.7/site-packages/sbi/neural_nets/flow.py:62: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  warn("In one-dimensional output space, this flow is limited to Gaussians")


 Neural network successfully converged after 22 epochs.

In [25]:
X_fid = np.median(control_data[:,1:], axis=0)
T_fid = np.array([0.5])
dT = np.array([0.2])

In [31]:
for i in range(10): 
    print(Cflow.CATE(X_fid, T_fid, dT, Nsample=10000, progress_bar=False))#, transf=lambda x: 10**x)

-0.015604734
-0.008635998
-0.014582157
-0.020490885
-0.008505821
-0.03550744
-0.023151636
-0.02051258
-0.005250931
-0.021917343
